# Automated ML

Below are the dependencies that will be needed to complete the project.

In [1]:
import azureml.core
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Webservice, Model
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


import pandas as pd
import sklearn
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, sep = '\n')

quick-starts-ws-132732
aml-quickstarts-132732
a0a76bad-11a1-4a2d-9887-97a29122c8ed


In [3]:
# choose a name for experiment
experiment_name = 'capstone-heart-failure-exp'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-heart-failure-exp,quick-starts-ws-132732,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
    
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_nodes=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True)         
    print("Cluster is ready")

Existing cluster detected, make use of it!


In [5]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir='./', target_path='dataset/')

Uploading an estimated of 8 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 8
Uploading ./endpoint.py
Uploaded ./endpoint.py, 2 files out of an estimated total of 8
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 3 files out of an estimated total of 8
Uploading ./score.py
Uploaded ./score.py, 4 files out of an estimated total of 8
Uploading ./train.py
Uploaded ./train.py, 5 files out of an estimated total of 8
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2021-0-1-9-14-26.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2021-0-1-9-14-26.ipynb, 6 files out of an estimated total of 8
Uploading ./.ipynb_aml_checkpoints/hyperparameter_tuning-checkpoint2021-0-1-9-15-5.ipynb
Uploaded ./.ipynb_aml_checkpoints/hyperparameter_tuning-checkpoint2021-0-1-9-15-5.ipynb, 7 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.

$AZUREML_DATAREFERENCE_fb993b057a944d7ba28c619803a00f4d

In [6]:
dataset = Dataset.get_by_name(ws, name="heart-failure")
print("Dataset is registered")
dataset

Dataset is registered


{
  "source": [
    "('workspaceblobstore', 'UI/01-01-2021_091159_UTC/heart_failure_clinical_records_dataset.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "dd6971b3-2e94-44b4-a4de-625490ca74dc",
    "name": "heart-failure",
    "version": 1,
    "description": "External data from kaggle",
    "workspace": "Workspace.create(name='quick-starts-ws-132732', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-132732')"
  }
}

In [7]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

The following code shows a basic example of creating an AutoMLConfig object and submitting an experiment for classification.
I chose the automl settings below because I wanted to specify the experiment type as classification. The classification experiment will be carried out using AUC weighted as the primary metric with experiment timeout minutes set to 30 minutes and 5 cross-validation folds with the maximum number of iterations that would be executed concurrently set to 4.
All of these settings defines the machine learning task.


The configuration object below contains and persists the parameters for configuring the experiment run, as well as the training data to be used at run time.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=new_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             enable_voting_ensemble= True,
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment

remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_ef15e003-9024-458d-bef1-ff584eb1739a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_ef15e003-9024-458d-bef1-ff584eb1739a',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-01T09:19:11.85318Z',
 'endTimeUtc': '2021-01-01T09:43:15.631352Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-heart-failure-exp","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-132732","workspace_name":"quick-starts-ws-132732","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: capstone-heart-failure-exp,
Id: AutoML_ef15e003-9024-458d-bef1-ff584eb1739a_52,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=1.0416666666666667,
                                                                                               reg_lambda=1.4583333333333335,
 

In [14]:
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name)
    print(metric)

f1_score_micro
0.8629378531073446
precision_score_macro
0.8563525276280952
average_precision_score_micro
0.9302751427379299
average_precision_score_weighted
0.9381112777032025
recall_score_micro
0.8629378531073446
recall_score_weighted
0.8629378531073446
balanced_accuracy
0.8362756090808418
log_loss
0.3953965133064009
AUC_weighted
0.9268350867478775
accuracy
0.8629378531073446
weighted_accuracy
0.8803598943707976
norm_macro_recall
0.6725512181616834
AUC_macro
0.9268350867478775
matthews_correlation
0.69066326054509
AUC_micro
0.9271391681828339
average_precision_score_macro
0.9176143083457242
precision_score_micro
0.8629378531073446
f1_score_weighted
0.8601186963038343
recall_score_macro
0.8362756090808418
precision_score_weighted
0.8743951425377446
f1_score_macro
0.836343911792978
confusion_matrix
aml://artifactId/ExperimentRun/dcid.AutoML_ef15e003-9024-458d-bef1-ff584eb1739a_52/confusion_matrix
accuracy_table
aml://artifactId/ExperimentRun/dcid.AutoML_ef15e003-9024-458d-bef1-ff584eb17

In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-heart-failure-exp,AutoML_ef15e003-9024-458d-bef1-ff584eb1739a_52,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(best_run.get_tags())

Best Run Id:  AutoML_ef15e003-9024-458d-bef1-ff584eb1739a_52

 Accuracy: 0.8629378531073446
{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":3,"CurrentNodeCount":4}', 'ensembled_iterations': '[33, 47, 12, 31, 28, 19, 43, 38]', 'ensembled_algorithms': "['RandomForest', 'RandomForest', 'RandomForest', 'RandomForest', 'RandomForest', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier']", 'ensemble_weights': '[0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.25, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333]', 'best_individual_pipeline_score': '0.9150766196013288', 'best_individual_iteration': '33', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True', 'model_explain_run_id': 'AutoML_ef15e003-9024-458d-bef1-ff584eb1739a_ModelExplain', 'model_explanation': 'True'}


In [18]:
#TODO: Save the best model

best_run.register_model(model_name='automl_model',model_path='/outputs',properties={'AUC_weighted':best_run_metrics['AUC_weighted']},tags={'Training context':'Auto ML'})

Model(workspace=Workspace.create(name='quick-starts-ws-132732', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-132732'), name=automl_model, id=automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC_weighted': '0.9268350867478775'})

In [19]:
model_name = best_run.properties['model_name']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
# Register the model

model = remote_run.register_model(model_name = model_name,
                                  description = 'Automl model')

print('Name:', model.name)

Name: AutoMLef15e003952


In [26]:
env = best_run.get_environment()

script_name = 'score.py'


best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [56]:
# Create an inference config

inference_config = InferenceConfig(entry_script= script_name, environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)
aci_service_name='capstone-service'


In [57]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
                        

In [58]:
service.wait_for_deployment(show_output=True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [59]:
%run endpoint.py

{"result": [1, 1]}


In [60]:
data_test = dataset.to_pandas_dataframe().dropna()
data_sample = data_test.sample(3)
y_true = data_sample.pop('DEATH_EVENT')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)

{"data": [{"age": 46.0, "anaemia": 1, "creatinine_phosphokinase": 291, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 0, "platelets": 348000.0, "serum_creatinine": 0.9, "serum_sodium": 140, "sex": 0, "smoking": 0, "time": 109}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 260, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 255000.0, "serum_creatinine": 2.2, "serum_sodium": 132, "sex": 0, "smoking": 1, "time": 45}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 1, "platelets": 127000.0, "serum_creatinine": 0.9, "serum_sodium": 145, "sex": 0, "smoking": 0, "time": 95}]}


TODO: In the cell below, print the logs of the web service and delete the service

In [61]:
output = service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

Prediction:  {"result": [0, 1, 0]}
True Values:  [0 1 0]


In [62]:
service.get_logs()

'2021-01-01T10:59:11,013593300+00:00 - gunicorn/run \n2021-01-01T10:59:11,015230800+00:00 - iot-server/run \n2021-01-01T10:59:11,040872500+00:00 - nginx/run \n2021-01-01T10:59:11,022283600+00:00 - rsyslog/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [54]:
# Delete the service
service.delete()

In [55]:
new_cluster.delete()